### Librerias Importantes

By Elías Jesús Ventura-Molina

Modelos en Keras: https://keras.io/api/models/    

Capas en Keras: https://keras.io/api/layers/    

Importar bancos de datos desde Keras: https://keras.io/api/datasets/      
El banco de datos Mnist contiene imagenes de 28x28 pixeles. 60,000 son de entrenamiento y 10,000 de prueba

In [5]:
# Para implementar un modelo secuencial, documentacion:
from tensorflow.keras.models import Sequential

# Capas que se utilizaran, si no se sabe que capas se utilizaran, importar todas 
from tensorflow.keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D, Dropout

# Importar un banco de datos desde Keras
from tensorflow.keras.datasets import mnist

# Optimizador
from tensorflow.keras.optimizers import RMSprop

# Keras sobre Tensorflow
from tensorflow.keras import backend as K

# Onehot -> Categorico
from tensorflow.keras.utils import to_categorical

# Matriz de confusion y reporte
from sklearn.metrics import classification_report, confusion_matrix

import numpy as np

### Parametros importantes

In [6]:
# Tamaño del lote de parametros que seran propagados en la red 
batch_size = 128

# Numero de clases
nb_classes = 10

# Numero de epocas
nb_epoch = 10

img_rows, img_cols = 28, 28         # Tamaño de las imagenes
pool_size = (2, 2)                  # tamaño del filtro de las capas de pooling
prob_drop_conv = 0.2                # Probabilidad de dropout en capas convolucionales
prob_drop_hidden = 0.5              # Probabilidad de dropout en capas densas

# Tamaño de entrada para la primer capa convolucional
input_shape = (img_rows, img_cols, 1)

In [8]:
# Cargar el banco de datos, los conjuntos de entrenamiento y prueba ya estan definidos
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print('X_train original shape:', X_train.shape)

# Backend
if K.image_data_format() == 'th':
    # Para Theano
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    # Para TensorFlow
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)



# Rasgos de todos los patrones de entrenamiento
X_train = X_train.astype('float32') / 255.

# Rasgos de todos los patrones de prueba
X_test = X_test.astype('float32') / 255.

# Clases de todos los patrones de entrenamiento
Y_train = to_categorical(y_train, nb_classes)
# Clases de todos los patrones de prueba

Y_test = to_categorical(y_test, nb_classes)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train original shape: (60000, 28, 28)
X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


### Modelo de la Red Neuronal Convolucional

En esta sección se define la arquitectura de la red neuronal convolucional.   

Tipos de modelos: https://www.pyimagesearch.com/2019/10/28/3-ways-to-create-a-keras-model-with-tensorflow-2-0-sequential-functional-and-model-subclassing/   

Una capa convolucional 2D utiliza filtros en dos dimensiones, una 3D utiliza filtros en tres dimensiones.    
Para imegenes RGB se pueden usar los dos tipos de capas, si se usa 2D, el filtro pasara por los tres canales de color con los mismos pesos, se se desea usar pesos especificos para cada canal de color, se debe usar una capa 3D.

Parametros que se pueden ajustar en una capa convolucional 2D: https://keras.io/api/layers/convolution_layers/convolution2d/

Parametros que se pueden ajustar en una capa MaxPooling: https://keras.io/api/layers/pooling_layers/max_pooling2d/

Parametros que se pueden ajustar en una capa Densa: https://keras.io/api/layers/core_layers/dense/

Optimizadores:    
¿Qué es un optimizador?: https://algorithmia.com/blog/introduction-to-optimizers    
Optimizadores en Keras: https://keras.io/api/optimizers/

Loss Function:    
¿Qué es y como escogerla?: https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/   
Loss Function en Keras: https://keras.io/api/losses/    

Padding: https://www.aprenderaprogramar.com/index.php?option=com_content&view=article&id=732:concepto-de-margen-y-relleno-css-diferencias-entre-margin-y-padding-css-box-model-ejemplos-cu01028d&catid=75&Itemid=203

In [9]:
# Definir cual es el tipo de modelo: 
model = Sequential()

# Primera Capa convolucional
model.add(Convolution2D(32, 3, 3, padding='same', activation='relu', input_shape=input_shape))
# El tamaño del filtro o kernel fue definido anteriormente
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Dropout(prob_drop_conv))

# Segunda capa convolucional
model.add(Convolution2D(64, 3, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Dropout(prob_drop_conv))

# Tercera capa convolucional
model.add(Convolution2D(128, 3, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
# Flatten es utilizado para "aplanar" la salida de las capas convolucionales, pasar de array
# multidimensional a unidimensional
model.add(Flatten())
model.add(Dropout(prob_drop_conv))

# Primer capa oculta densa
# Se dice que es capa oculta y no de entrada porque Keras crea automaticamente la capa densa de
# entrada basado en la salida de Flatten
model.add(Dense(625, activation='relu'))
model.add(Dropout(prob_drop_hidden))

# Segunda capa saida densa
model.add(Dense(10, activation='softmax'))

# Optimizador
opt = RMSprop(lr=0.001, rho=0.9)

# Compilar el modelo
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Sirve para ver de forma general la arquitectura de la red y conocer el numero de parametros
#generados
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 10, 10, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 5, 5, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 2, 64)          18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 128)         7

### Entrenamiento

En esta fase se pueden usar las clases en forma One hot o en forma categorica.
Convertirlos a forma categorica para poder hacer la matriz de confusión

In [10]:
train_labels = to_categorical(Y_train)
test_labels = to_categorical(Y_test)

history = model.fit(X_train, Y_train, epochs=nb_epoch, batch_size=batch_size, shuffle=True, verbose=1)


Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 9s 149us/sample - loss: 0.8595 - accuracy: 0.7034
Epoch 2/10
60000/60000 [==============================] - 2s 34us/sample - loss: 0.4133 - accuracy: 0.8675
Epoch 3/10
60000/60000 [==============================] - 2s 34us/sample - loss: 0.3151 - accuracy: 0.9005
Epoch 4/10
60000/60000 [==============================] - 2s 34us/sample - loss: 0.2654 - accuracy: 0.9166
Epoch 5/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.2360 - accuracy: 0.9254
Epoch 6/10
60000/60000 [==============================] - 2s 34us/sample - loss: 0.2189 - accuracy: 0.9307
Epoch 7/10
60000/60000 [==============================] - 2s 34us/sample - loss: 0.2099 - accuracy: 0.9345
Epoch 8/10
60000/60000 [==============================] - 2s 34us/sample - loss: 0.2003 - accuracy: 0.9380
Epoch 9/10
60000/60000 [==============================] - 2s 34us/sample - loss: 0.1927 - accuracy: 0.9397
Epoch 10/10
6

### Prueba

model.evaluate predice la salida y calcula la metrica especificada en model.compile    
model.predict solo predice la salida

In [11]:
evaluation = model.evaluate(X_test, Y_test, batch_size=256, verbose=1)
print('Accuracy: %.3f' % (evaluation[1]))

10000/10000 [==============================] - 0s 34us/sample - loss: 0.1018 - accuracy: 0.9673
Accuracy: 0.967


### Matriz de confusión & Reporte

Se necesita tener las clases en forma categorica

In [12]:
rounded_predictions = model.predict_classes(X_test, batch_size=256, verbose=0)
rounded_labels = np.argmax(Y_test, axis=1)

cm = confusion_matrix(rounded_labels, rounded_predictions)
print('                  Matriz de Confusion')
print('\n')
print(cm)

print('\n')
print('\n')
print('\n')
print('                      Reporte de Clasificacion')
print('\n')
target_names = ['1', '2', '3', '4','5','6','7','8','9','10']
print(classification_report(rounded_labels, rounded_predictions, target_names=target_names))

                  Matriz de Confusion


[[ 967    0    0    0    2    2    2    1    6    0]
 [   0 1119    2    4    1    1    3    0    5    0]
 [   9    0 1001    2    3    0    0    7    9    1]
 [   0    2    4  941    0   26    1    7   26    3]
 [   0    1    0    0  959    0    8    2    2   10]
 [   5    0    0    7    0  860    9    0    9    2]
 [   4    3    0    0    4    3  941    0    3    0]
 [   2    7   12    3    5    0    0  986    4    9]
 [   5    1    2    2    5    6    4    1  941    7]
 [   6    3    1    3   14    5    1    8   10  958]]






                      Reporte de Clasificacion


              precision    recall  f1-score   support

           1       0.97      0.99      0.98       980
           2       0.99      0.99      0.99      1135
           3       0.98      0.97      0.97      1032
           4       0.98      0.93      0.95      1010
           5       0.97      0.98      0.97       982
           6       0.95      0.96      0.96      